<p style="text-align: center;font-size:50px;">M. Ammar Siddique <br> BSDSF22M041</p>


<p style="text-align: center;font-size:50px;">Data Mining and Machine Learning </p>


<p style="text-align: center;font-size:50px;">Assignment # 3 </p>

                                                         
<p style="text-align: center;font-size:40px;">Zomato data set </p>

## Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Load dataset
df = pd.read_csv("zomato_data.csv")

# Drop unnecessary column
df.drop(columns=["name"], inplace=True)

# Convert categorical variables to numerical
df["online_order"] = df["online_order"].map({"Yes": 1, "No": 0})  # Binary Mapping
df["book_table"] = df["book_table"].map({"Yes": 1, "No": 0})  # Binary Mapping

# Convert 'approx_cost(for two people)' to numeric (removing commas)
df["approx_cost(for two people)"] = df["approx_cost(for two people)"].astype(str).str.replace(",", "").astype(float)

# Handle missing values by filling with median
df.fillna(df.median(numeric_only=True), inplace=True)

# One-Hot Encoding for 'listed_in(type)' (removes ordinal relationship)
df = pd.get_dummies(df, columns=["listed_in(type)"], drop_first=True)

# Define independent (X) and dependent variable (y)
X = df.drop(columns=["rate"])  # Features
y = df["rate"]  # Target

# Split data into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data Preprocessing Completed!")
print("X_train Shape:", X_train.shape)
print("X_test Shape:", X_test.shape)


In [ ]:
df.head(25)

In [ ]:
# Check data types
print("Before Fixing:\n", X_train.dtypes)

# Convert categorical variables to numeric using One-Hot Encoding
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Ensure data types are numeric
X_train = X_train.apply(pd.to_numeric)
X_test = X_test.apply(pd.to_numeric)

# Check again after conversion
print("After Fixing:\n", X_train.dtypes)


In [12]:
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


In [14]:
import pandas as pd

def clean_rate(rate):
    if isinstance(rate, str):  # Check if value is string
        if '/' in rate:
            return float(rate.split('/')[0])  # Extract the first part (e.g., '3.3' from '3.3/5')
        else:
            return None  # Handle cases like 'NEW' or '-'
    return rate  # If already numeric, return as is

y_train = y_train.apply(clean_rate)
y_train = y_train.astype(float)  # Convert to float


In [15]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(float)


# Forward elimination

In [16]:
import statsmodels.api as sm

# Start with an empty model
X_selected = pd.DataFrame()  # Initially empty
remaining_features = list(X_train.columns)

best_features = []  # Store selected features

while remaining_features:
    scores = {}
    
    for feature in remaining_features:
        # Add one feature at a time
        X_temp = X_selected.copy()
        X_temp[feature] = X_train[feature]
        X_temp = sm.add_constant(X_temp)  # Add constant term (intercept)
        
        model = sm.OLS(y_train, X_temp).fit()  # Fit model
        
        scores[feature] = model.pvalues[feature]  # Store p-value

    # Find the feature with the lowest p-value
    best_feature = min(scores, key=scores.get)
    
    if scores[best_feature] < 0.05:  # If p-value is significant, add feature
        best_features.append(best_feature)
        X_selected[best_feature] = X_train[best_feature]
        remaining_features.remove(best_feature)
    else:
        break  # Stop if no more significant features

# Final model with selected features
X_selected = sm.add_constant(X_selected)
final_model = sm.OLS(y_train, X_selected).fit()

# Evaluate on test data
X_test_selected = X_test[best_features]
X_test_selected = sm.add_constant(X_test_selected)
y_pred = final_model.predict(X_test_selected)

# Print results
print("Selected Features:", best_features)
print(final_model.summary())


Selected Features: ['votes', 'online_order']
                            OLS Regression Results                            
Dep. Variable:                   rate   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     36.98
Date:                Thu, 06 Mar 2025   Prob (F-statistic):           3.97e-13
Time:                        11:32:20   Log-Likelihood:                -24.701
No. Observations:                 118   AIC:                             55.40
Df Residuals:                     115   BIC:                             63.71
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
con

# Backward elimination

In [19]:
import pandas as pd
import statsmodels.api as sm

# Load your dataset
df = pd.read_csv('zomato_data.csv')

# Select relevant columns
features = ['votes', 'online_order', 'book_table', 'approx_cost(for two people)']
target = 'rate'

# Convert categorical variables if necessary
df['online_order'] = df['online_order'].map({'Yes': 1, 'No': 0})
df['book_table'] = df['book_table'].map({'Yes': 1, 'No': 0})

# Extract numeric part from rate (handling 'NEW' and '-')
df['rate'] = df['rate'].astype(str).str.extract(r'(\d+\.\d+)')  # Extract decimal numbers
df['rate'] = df['rate'].astype(float)  # Convert to float

# Drop NaN values
df = df.dropna()

# Define X and y
X = df[features]
y = df[target]

# Add constant (intercept)
X = sm.add_constant(X)

# Perform Backward Elimination
def backward_elimination(X, y, significance_level=0.05):
    while True:
        model = sm.OLS(y, X).fit()
        p_values = model.pvalues[1:]  # Exclude constant
        max_p_value = p_values.max()
        
        if max_p_value > significance_level:
            worst_feature = p_values.idxmax()
            print(f"Dropping '{worst_feature}' (p-value: {max_p_value:.4f})")
            X = X.drop(columns=[worst_feature])
        else:
            break
            
    return X, model

# Run Backward Elimination
X_selected, final_model = backward_elimination(X, y)

# Print final summary
print("\nFinal Model Summary:")
print(final_model.summary())


Dropping 'approx_cost(for two people)' (p-value: 0.7842)

Final Model Summary:
                            OLS Regression Results                            
Dep. Variable:                   rate   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                     26.46
Date:                Thu, 06 Mar 2025   Prob (F-statistic):           1.08e-13
Time:                        11:38:33   Log-Likelihood:                -42.235
No. Observations:                 148   AIC:                             92.47
Df Residuals:                     144   BIC:                             104.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------

# Bidirectional Elimination

In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load dataset
df = pd.read_csv("zomato_data.csv")

# Convert categorical variables
df['online_order'] = df['online_order'].map({'Yes': 1, 'No': 0})
df['book_table'] = df['book_table'].map({'Yes': 1, 'No': 0})

# Convert 'rate' column to numeric
df['rate'] = df['rate'].str.replace('/5', '', regex=True).replace({'NEW': None, '-': None}).astype(float)

# Handle 'approx_cost(for two people)' column
if df['approx_cost(for two people)'].dtype == 'O':  # Check if it's a string
    df['approx_cost(for two people)'] = df['approx_cost(for two people)'].astype(str).str.replace(',', '', regex=True)

df['approx_cost(for two people)'] = pd.to_numeric(df['approx_cost(for two people)'], errors='coerce')
df = df.dropna()  # Drop NaN values

# Define independent variables (X) and dependent variable (y)
features = ['votes', 'online_order', 'book_table', 'approx_cost(for two people)']
X = df[features]
y = df['rate']

# Add constant (intercept)
X = sm.add_constant(X)

# Bidirectional elimination function
def bidirectional_elimination(X, y, significance_level=0.05):
    selected_vars = list(X.columns)
    
    while True:
        X_opt = X[selected_vars]
        model = sm.OLS(y, X_opt).fit()
        p_values = model.pvalues
        
        # Backward Elimination step
        max_p = p_values.max()
        if max_p > significance_level:
            excluded_var = p_values.idxmax()
            selected_vars.remove(excluded_var)
            continue

        # Forward Selection step (check if any excluded variable should be added back)
        excluded_vars = list(set(X.columns) - set(selected_vars))
        added = False
        for var in excluded_vars:
            temp_vars = selected_vars + [var]
            temp_model = sm.OLS(y, X[temp_vars]).fit()
            if temp_model.pvalues[var] < significance_level:
                selected_vars.append(var)
                added = True

        if not added:
            break

    return sm.OLS(y, X[selected_vars]).fit()

# Run bidirectional elimination
model = bidirectional_elimination(X, y)

# Print final summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                   rate   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                     26.46
Date:                Thu, 06 Mar 2025   Prob (F-statistic):           1.08e-13
Time:                        11:44:59   Log-Likelihood:                -42.235
No. Observations:                 148   AIC:                             92.47
Df Residuals:                     144   BIC:                             104.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            3.4684      0.035    100.342   

# All Columns

In [24]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load dataset
df = pd.read_csv("zomato_data.csv")

# Convert categorical variables
df['online_order'] = df['online_order'].map({'Yes': 1, 'No': 0})
df['book_table'] = df['book_table'].map({'Yes': 1, 'No': 0})

# Convert 'rate' column to numeric
df['rate'] = df['rate'].str.replace('/5', '', regex=True).replace({'NEW': None, '-': None}).astype(float)

# Handle 'approx_cost(for two people)' column
if df['approx_cost(for two people)'].dtype == 'O':  # Check if it's a string
    df['approx_cost(for two people)'] = df['approx_cost(for two people)'].astype(str).str.replace(',', '', regex=True)

df['approx_cost(for two people)'] = pd.to_numeric(df['approx_cost(for two people)'], errors='coerce')
df = df.dropna()  # Drop NaN values

# Define independent variables (X) and dependent variable (y)
features = ['votes', 'online_order', 'book_table', 'approx_cost(for two people)']
X = df[features]
y = df['rate']

# Add constant (intercept)
X = sm.add_constant(X)

# Fit multiple linear regression model with NO elimination
model = sm.OLS(y, X).fit()

# Print model summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                   rate   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.338
Method:                 Least Squares   F-statistic:                     19.74
Date:                Thu, 06 Mar 2025   Prob (F-statistic):           5.90e-13
Time:                        11:46:42   Log-Likelihood:                -42.196
No. Observations:                 148   AIC:                             94.39
Df Residuals:                     143   BIC:                             109.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             